In [1]:
import re
from pathlib import Path
import pandas as pd
import pypsa
import numpy as np
import re
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

### 1. Import network components

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ScenarioConfig:
    name: str
    inputs_dir: Path         # folder containing buses.csv, powerplant_units.csv, ...
    nc_path: Path            # redispatch_full_redispatch.nc


#### 1.Example usage: Select scenario configuration

In [11]:
# Example scenario configurations

# 1. Example 06a - with storage & without transmission expansion
cfg_06a_stor = ScenarioConfig(
    name="example_06a_storage",
    inputs_dir=Path(r"C:\Users\par19744\Python_projects\ASSUME_project\ASSUME_Use_case\2035\assume\examples\inputs\example_06a"),
    nc_path=Path(r"C:\Users\par19744\Python_projects\ASSUME_project\ASSUME_results\Use case results\O45 Strom\with storage\example_06a_base\pypsa_nc\redispatch_full_redispatch.nc"),
)

"""
# 2. Example 06a - with storage & with transmission expansion
cfg_06a_stor_trans = ScenarioConfig(
    name="example_06a_storage_transmission",
    inputs_dir=Path(r"C:\Users\par19744\Python_projects\ASSUME_project\ASSUME_Use_case\2035\assume\examples\inputs\example_06a"),
    nc_path=Path(r"C:\Users\par19744\Python_projects\ASSUME_project\ASSUME_results\Use case results\O45 Strom\with_storage_transmission\example_06a_base\pypsa_nc\redispatch_full_redispatch.nc"),
)
"""

In [5]:
# technology harmonization rules (keep in one place)
POWERPLANT_TECH_MAP = {
    "open cycle gas turbine": "natural gas",
    "gas steam turbine": "natural gas",
    "combined cycle gas turbine": "natural gas",
    "gas engine": "natural gas",
    "small gas mix": "natural gas",
}

def load_scenario_inputs(cfg: ScenarioConfig) -> dict:
    """
    Returns:
      - buses: DataFrame with at least ['bus','bus_id','x','y',...]
      - units: Series mapping unit_id -> technology  (index=unit_id)
      - raw: optional raw dfs if you want them
    """
    p = cfg.inputs_dir

    # --- buses ---
    buses = pd.read_csv(p / "buses.csv")
    # your file uses 'name' -> bus
    if "name" in buses.columns and "bus" not in buses.columns:
        buses = buses.rename(columns={"name": "bus"})
    # ensure types are stable for joins
    if "bus_id" in buses.columns:
        buses["bus_id"] = pd.to_numeric(buses["bus_id"], errors="coerce")

    # --- powerplants ---
    powerplants = pd.read_csv(p / "powerplant_units.csv")
    powerplants = powerplants[["name", "technology"]].rename(columns={"name": "unit_id"})
    powerplants["technology"] = powerplants["technology"].replace(POWERPLANT_TECH_MAP)

    # --- storage ---
    storage_units = pd.read_csv(p / "storage_units.csv")
    storage_units = storage_units[["name", "technology"]].rename(columns={"name": "unit_id"})

    # --- exchange ---
    exchange_units = pd.read_csv(p / "exchange_units.csv")
    exchange_units = exchange_units.rename(columns={"name": "unit_id"})
    exchange_units["technology"] = "exchange_units"
    exchange_units = exchange_units[["unit_id", "technology"]]

    # --- combine ---
    units_df = pd.concat([powerplants, storage_units, exchange_units], ignore_index=True)

    # if duplicate unit_id appears, last wins; you can change policy if needed
    units = units_df.drop_duplicates("unit_id", keep="last").set_index("unit_id")["technology"]

    return {
        "buses": buses,
        "units": units,                  # Series: unit_id -> tech
        "units_df": units_df,            # optional
        "powerplants": powerplants,
        "storage_units": storage_units,
        "exchange_units": exchange_units,
    }


### 2. Redispatch Visualization

In [6]:
def _base_unit_id(gen_name: str) -> str:
    """
    Strip common suffixes added by redispatch/backups.
    Extend this if your naming differs.
    """
    return re.sub(r"(_up|_down)$", "", str(gen_name))

def load_redispatch_summary(
    nc_path: str | Path,
    units: pd.DataFrame | pd.Series | None = None,
    tech_col: str = "technology",
    storage_techs: set[str] | None = None,
) -> dict:
    """
    Loads a PyPSA netcdf and returns:
      - n: pypsa.Network
      - summary: time series totals and storage split
      - up_by_tech, dn_by_tech: time series per technology (if units provided)
    """

    nc_path = Path(nc_path)
    n = pypsa.Network()
    n.import_from_netcdf(str(nc_path))

    # --- split generators_t.p into up/down by column naming convention ---
    gen_p = n.generators_t.p.copy()
    up_cols = [c for c in gen_p.columns if str(c).endswith("_up")]
    dn_cols = [c for c in gen_p.columns if str(c).endswith("_down")]

    up = gen_p[up_cols] if up_cols else pd.DataFrame(index=n.snapshots)
    dn = gen_p[dn_cols] if dn_cols else pd.DataFrame(index=n.snapshots)

    up_total = up.sum(axis=1)
    dn_total = dn.sum(axis=1)

    # --- attach technology mapping (optional but recommended) ---
    up_by_tech = None
    dn_by_tech = None
    up_storage = pd.Series(0.0, index=n.snapshots)
    dn_storage = pd.Series(0.0, index=n.snapshots)

    if storage_techs is None:
        storage_techs = {"PSPP", "li_ion_battery", "compressed_air_storage"}

    if units is not None:
        # units can be a DataFrame with column 'technology' indexed by unit_id,
        # OR a Series mapping unit_id -> technology
        if isinstance(units, pd.DataFrame):
            tech_map = units[tech_col]
        else:
            tech_map = units  # Series

        n.generators["base_unit_id"] = n.generators.index.to_series().map(_base_unit_id)
        n.generators["technology"] = n.generators["base_unit_id"].map(tech_map)

        tech = n.generators["technology"]

        # group by technology, dropping columns where technology is NaN
        up_by_tech = up.groupby(tech.reindex(up.columns), axis=1).sum() if not up.empty else pd.DataFrame(index=n.snapshots)
        dn_by_tech = dn.groupby(tech.reindex(dn.columns), axis=1).sum() if not dn.empty else pd.DataFrame(index=n.snapshots)

        # storage split
        if not up_by_tech.empty:
            up_storage = up_by_tech.loc[:, up_by_tech.columns.intersection(storage_techs)].sum(axis=1)
        if not dn_by_tech.empty:
            dn_storage = dn_by_tech.loc[:, dn_by_tech.columns.intersection(storage_techs)].sum(axis=1)

    summary = pd.DataFrame(
        {
            "up_total_MW": up_total,
            "down_total_MW": dn_total,
            "up_storage_MW": up_storage,
            "down_storage_MW": dn_storage,
            "up_non_storage_MW": up_total - up_storage,
            "down_non_storage_MW": dn_total - dn_storage,
        },
        index=n.snapshots,
    )

    return {
        "network": n,
        "summary": summary,
        "up_by_tech": up_by_tech,
        "dn_by_tech": dn_by_tech,
        "path": str(nc_path),
    }


In [7]:
def run_scenario(cfg: ScenarioConfig, storage_techs=None) -> dict:
    inputs = load_scenario_inputs(cfg)
    redispatch = load_redispatch_summary(cfg.nc_path, units=inputs["units"], storage_techs=storage_techs)

    return {
        "cfg": cfg,
        "inputs": inputs,
        "redispatch": redispatch,
    }


#### 2.Example usage: Redispatch volumes

In [8]:
res_06a = run_scenario(cfg_06a)
res_06a["redispatch"]["summary"].head()

Index(['demand_CRM_pos', 'demand_CRM_neg'], dtype='object', name='name')
INFO:pypsa.io:Imported network redispatch_full_redispatch.nc has buses, carriers, generators, lines, loads
C:\Users\par19744\AppData\Local\Temp\ipykernel_33500\2902291634.py:59: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

C:\Users\par19744\AppData\Local\Temp\ipykernel_33500\2902291634.py:60: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.



,up_total_MW,down_total_MW,up_storage_MW,down_storage_MW,up_non_storage_MW,down_non_storage_MW
snapshot,,,,,,
0,5900.964592,89805.931548,4923.443873,3414.837062,977.520719,86391.094487
1,871.588514,96757.475770,12.981683,0.000000,858.606832,96757.475770
2,20444.556786,25019.851136,18655.952959,2130.451879,1788.603827,22889.399256
3,25752.578569,25752.578569,22193.207279,3779.572696,3559.371290,21973.005873
4,26552.972893,26552.972893,23559.669620,1874.785228,2993.303273,24678.187665
